In [1]:
%load_ext autoreload
%autoreload 2
%matplotlib inline  
from IPython.display import display, HTML, SVG
from db import Result
import papermill as pm
import os
import seaborn
import pandas
import matplotlib.pyplot as plt
from collections import defaultdict 
from sklearn.metrics import auc, precision_recall_curve, roc_curve, confusion_matrix, roc_auc_score, roc_curve, f1_score, accuracy_score
from sklearn.metrics import cohen_kappa_score
from evaluate import calculate_confusion_matrix_stats_predictions, calculate_roc_curve, roc_auc_score
from scipy.stats import binom_test, fisher_exact, chi2_contingency
from tabulate import tabulate
from calculate_features import all_features
from config import config
from data import data
plt.rcParams['svg.fonttype'] = 'none'

Using TensorFlow backend.


In [2]:
SKIP = False
UUIDS = [    
#    "7bd2f246-f096-4dfb-86ba-2abb4fa37d4d", # features
#    "31e84705-3483-406d-9699-026d3367f31f", # t2
#    "b776df8a-f966-4afd-bf7a-f2b4c1fe3a1c", # t1
#    "01e93f79-3d49-4809-8dab-fa2055724377", # backup t1
#    "5d1d4156-ee5a-49ff-87cf-ad4a11be8aef", # t1
#    "47cf8213-5aca-4e1f-b2ba-8dbf4aac7830", # t2
#    "b0fb56eb-f2fa-4a5a-bd76-297ec64a371e", # features
#    "02aa461f-be56-4138-9fcd-084a56bab732", # t2
#    "cdbba6a0-73b3-4ef8-8ea5-1a5f64525320", # t2    
#     "405a845e-febd-4a81-a864-72d10f45c757", # t2 overfit
#     "95b693b8-6ab8-4862-ae6a-404d33751327", # t1 nice sensitivity and specificity balance
#     "d2ccc80f-35f2-4477-943e-708e48c518a4", # t1 not great 0.66 validation acc
#     "79e0e7d4-4250-4935-abfa-c4bbccc19748", # t2 not great 0.66 validation acc but works with ^ with less correlation
#     "40b2f9eb-44cc-4fef-9580-e560ceac8315", # t1 overfit
#     "75683443-233d-4f78-ae9f-f7c6ed085611", # t1, 
#     "a063f816-ae0e-42ab-92aa-f00092cec894", # features    

#     "59a36cd3-40b9-479f-9764-b7d8223264f7", # t2
#     "ab1a810d-5a82-434d-8ecd-28459d7dd137", # t1
     "ebe1e4e9-fd1a-44c5-b06a-6c77579b5be5", # features
#    "c798ed37-3dbc-4515-9559-f2d04d42d679", # t2 by auc
#     "1140a993-9934-45ee-b354-c3c778a4ff61", # t1 by validation specificity
#     "bbcee945-5abb-4d11-8035-68b8a3d484d4", # t2 by validation sensitivity
#     "bcae9141-60d6-4e54-b4d6-af01549681bb", # t2 by validation specificity
#     "86e91ce1-e2ba-4726-bfad-e68ea926e0f7", # t1 by validation sensitivity
     "59a36cd3-40b9-479f-9764-b7d8223264f7", # t2
     "acdd5165-089b-48fd-a936-af404577f7cf", # t1
]
SELECTIONS = {
    # old selections
    "9adb8e80-457c-4419-8b01-92325ba99fc3": { # t1
        ("benign", "benign"): ["Kidney-Mayo-060"],
        ("benign", "malignant"): ["Kidney-HP-049"],
        ("malignant", "malignant"): ["Kidney-Penn-082"],
        ("malignant", "benign"): [],
    },
    "391b0942-7801-4184-b551-2b62fe269c58": { # t2
        ("benign", "benign"): ["Kidney-Mayo-055", "Kidney-Mayo-054"],
        ("benign", "malignant"): ["Kidney-Mayo-046", "Kidney-Penn-004"],
        ("malignant", "malignant"): ["Kidney-TCGA-034", "Kidney-Penn-108"],
        ("malignant", "benign"): ["Kidney-Penn-193", "Kidney-XY2-007"],
    },
}
TSNE_PERPLEXITY = {
    "acdd5165-089b-48fd-a936-af404577f7cf": 10, # t1
    "1140a993-9934-45ee-b354-c3c778a4ff61": 10, # t1 by validation specificity
    "bbcee945-5abb-4d11-8035-68b8a3d484d4": 10, # t2 by validation sensitivity
    "bcae9141-60d6-4e54-b4d6-af01549681bb": 10, # t2 by validation specificity
    "86e91ce1-e2ba-4726-bfad-e68ea926e0f7": 10, # t1 by validation sensitivity    
    "bbcee945-5abb-4d11-8035-68b8a3d484d4": 10, # t2 by validation sensitivity    ""
    "1140a993-9934-45ee-b354-c3c778a4ff61": 10, # t1 by test specificity
    "59a36cd3-40b9-479f-9764-b7d8223264f7": 10, # t2
    "ab1a810d-5a82-434d-8ecd-28459d7dd137": 10, # t1    
    "75683443-233d-4f78-ae9f-f7c6ed085611": 10, # t1
    "40b2f9eb-44cc-4fef-9580-e560ceac8315": 10, # t1 overfit
    "d2ccc80f-35f2-4477-943e-708e48c518a4": 10, # t1 not great 0.66 validation acc
    "79e0e7d4-4250-4935-abfa-c4bbccc19748": 20, # t2 not great 0.66 validation acc but works with ^ with less correlation
    "405a845e-febd-4a81-a864-72d10f45c757": 20, # t2    
    "cdbba6a0-73b3-4ef8-8ea5-1a5f64525320": 20, # t2
    "02aa461f-be56-4138-9fcd-084a56bab732": 20, # t2
    "95b693b8-6ab8-4862-ae6a-404d33751327": 10, # t1
    "612a2e47-7f2c-484e-84d1-1f74c5f583d4": 20, # t2
    # old
    "31e84705-3483-406d-9699-026d3367f31f": 10, # t2
    "b776df8a-f966-4afd-bf7a-f2b4c1fe3a1c": 20, # t1
    # old
    "5d1d4156-ee5a-49ff-87cf-ad4a11be8aef": 20, # t1
    "47cf8213-5aca-4e1f-b2ba-8dbf4aac7830": 10, # t2
     "52858e66-1678-434a-92e4-bc18049cc233": 10, # t2
     "bc5794f0-5f96-4094-aff6-eab3c22b816a": 20, # t1
    
}
MODALITY = {
     "acdd5165-089b-48fd-a936-af404577f7cf": "t1", # t1
     "59a36cd3-40b9-479f-9764-b7d8223264f7": "t2", # t2    
     "ebe1e4e9-fd1a-44c5-b06a-6c77579b5be5": "features", # features"
}
old_modality = {
     "1140a993-9934-45ee-b354-c3c778a4ff61": "t1-specific", # t1 by validation specificity
     "bbcee945-5abb-4d11-8035-68b8a3d484d4": "t2-sensitive", # t2 by validation sensitivity
     "bcae9141-60d6-4e54-b4d6-af01549681bb": "t2-specific", # t2 by validation specificity
     "86e91ce1-e2ba-4726-bfad-e68ea926e0f7": "t1-sensitive", # t1 by validation sensitivity        
     "1140a993-9934-45ee-b354-c3c778a4ff61": "t1", # t1 by test specificity    
     "59a36cd3-40b9-479f-9764-b7d8223264f7": "t2", # t2
     "ab1a810d-5a82-434d-8ecd-28459d7dd137": "t1", # t1   

     "52858e66-1678-434a-92e4-bc18049cc233": "t2", # t2
     "bc5794f0-5f96-4094-aff6-eab3c22b816a": "t1", # t1
     "6a252ffb-a885-4ace-abee-95064928b494": "features", # features    
    "75683443-233d-4f78-ae9f-f7c6ed085611": "t1", 
    "40b2f9eb-44cc-4fef-9580-e560ceac8315": "t1", # t1 overfit
    "d2ccc80f-35f2-4477-943e-708e48c518a4": "t1", # t1 not great 0.66 validation acc
    "79e0e7d4-4250-4935-abfa-c4bbccc19748": "t2", # t2 not great 0.66 validation acc but works with ^ with less correlation    
    "405a845e-febd-4a81-a864-72d10f45c757": "t2", # t2        
    "cdbba6a0-73b3-4ef8-8ea5-1a5f64525320": "t2", # t2
    "02aa461f-be56-4138-9fcd-084a56bab732": "t2", # t2
    "95b693b8-6ab8-4862-ae6a-404d33751327": "t1", # t1
    "612a2e47-7f2c-484e-84d1-1f74c5f583d4": "t2", # t2
    "a063f816-ae0e-42ab-92aa-f00092cec894": "features", # features
    # old
    "7bd2f246-f096-4dfb-86ba-2abb4fa37d4d": "features",
    "31e84705-3483-406d-9699-026d3367f31f": "t2",
    "b776df8a-f966-4afd-bf7a-f2b4c1fe3a1c": "t1", 
    # old
    "5d1d4156-ee5a-49ff-87cf-ad4a11be8aef": "t1", # t1
    "47cf8213-5aca-4e1f-b2ba-8dbf4aac7830": "t2", # t2
    "b0fb56eb-f2fa-4a5a-bd76-297ec64a371e": "features", # features    
    # old
    "9adb8e80-457c-4419-8b01-92325ba99fc3": "t1", # t1
    "391b0942-7801-4184-b551-2b62fe269c58": "t2", # t2
    "dce56921-62a8-40f3-8cce-73b06d2b049c": "features", # features
}

In [3]:
import math
def adjusted_wald(p, n, z=1.96):
    p_adj = (n * p + (z**2)/2)/(n+z**2)
    n_adj = n + z**2
    span = z * math.sqrt(p_adj*(1-p_adj)/n_adj)
    return max(0, p_adj - span), min(p_adj + span, 1.0)

In [4]:
def execute_notebook(uuid, selections=None, tsne_perplexity=5, input_form="none", description=""): 
    name = "evaluate-{}-{}-{}.ipynb".format(description, input_form, uuid)
    if not SKIP and not os.path.exists(name): 
        pm.execute_notebook(
            "evaluate-specific-model.ipynb",
            "evaluate-{}-{}-{}.ipynb".format(description, input_form, uuid),
            parameters = dict(
                UUID=uuid,
                SELECTIONS=repr(selections),
                TSNE_PERPLEXITY=tsne_perplexity,
            ),
    )
    return name

In [5]:
completed_notebooks = list()
modality_by_notebook = dict()

In [6]:
for uuid in UUIDS: 
    print(uuid)
    result = Result.query.filter(Result.uuid == uuid).first()
    name = execute_notebook(uuid, SELECTIONS.get(uuid), TSNE_PERPLEXITY.get(uuid), result.input_form, result.description)
    completed_notebooks.append(name)
    modality_by_notebook[name] = MODALITY[uuid]
    print("done")

Input Notebook:  evaluate-specific-model.ipynb
Output Notebook: evaluate-round-9-t1-acdd5165-089b-48fd-a936-af404577f7cf.ipynb


ebe1e4e9-fd1a-44c5-b06a-6c77579b5be5
done
59a36cd3-40b9-479f-9764-b7d8223264f7
done
acdd5165-089b-48fd-a936-af404577f7cf



done


In [7]:
modality_by_notebook

{'evaluate-round-9-features-ebe1e4e9-fd1a-44c5-b06a-6c77579b5be5.ipynb': 'features',
 'evaluate-randomized-1-t2-59a36cd3-40b9-479f-9764-b7d8223264f7.ipynb': 't2',
 'evaluate-round-9-t1-acdd5165-089b-48fd-a936-af404577f7cf.ipynb': 't1'}

In [19]:
if not SKIP: 
    pm.execute_notebook(
        "evaluate-ensemble.ipynb",
        "evaluate-ensemble-{}.ipynb".format("-".join(UUIDS)),
        parameters = dict(
            MODELS=UUIDS,
            SCORE="accuracy",
        ),
    )
completed_notebooks.append("evaluate-ensemble-{}.ipynb".format("-".join(UUIDS)))

Input Notebook:  evaluate-ensemble.ipynb
Output Notebook: evaluate-ensemble-ebe1e4e9-fd1a-44c5-b06a-6c77579b5be5-59a36cd3-40b9-479f-9764-b7d8223264f7-acdd5165-089b-48fd-a936-af404577f7cf.ipynb


In [20]:
modality_by_notebook["evaluate-ensemble-{}.ipynb".format("-".join(UUIDS))] = "ensemble"

In [21]:
def get_notebook_output(notebook, name): 
    return notebook.dataframe[notebook.dataframe.name==name].value.tolist()[0]

In [22]:
train = dict()
validation = dict()
test = dict()
test_1 = dict()
test_2 = dict()
for notebook in completed_notebooks: 
    nb = pm.read_notebook(notebook)
    train["{}-{}".format(modality_by_notebook[notebook], "labels")] = get_notebook_output(nb, "train_labels")
    train["{}-{}".format(modality_by_notebook[notebook], "predictions")] = get_notebook_output(nb, "train_predictions")
    train["{}-{}".format(modality_by_notebook[notebook], "probabilities")] = get_notebook_output(nb, "train_probabilities")
    validation["{}-{}".format(modality_by_notebook[notebook], "labels")] = get_notebook_output(nb, "validation_labels")
    validation["{}-{}".format(modality_by_notebook[notebook], "predictions")] = get_notebook_output(nb, "validation_predictions")
    validation["{}-{}".format(modality_by_notebook[notebook], "probabilities")] = get_notebook_output(nb, "validation_probabilities")
    test["{}-{}".format(modality_by_notebook[notebook], "labels")] = get_notebook_output(nb, "test_labels")
    test["{}-{}".format(modality_by_notebook[notebook], "predictions")] = get_notebook_output(nb, "test_predictions")
    test["{}-{}".format(modality_by_notebook[notebook], "probabilities")] = get_notebook_output(nb, "test_probabilities")
    test_1["{}-{}".format(modality_by_notebook[notebook], "labels")] = get_notebook_output(nb, "test-1_labels")
    test_1["{}-{}".format(modality_by_notebook[notebook], "predictions")] = get_notebook_output(nb, "test-1_predictions")
    test_1["{}-{}".format(modality_by_notebook[notebook], "probabilities")] = get_notebook_output(nb, "test-1_probabilities")
    test_2["{}-{}".format(modality_by_notebook[notebook], "labels")] = get_notebook_output(nb, "test-2_labels")
    test_2["{}-{}".format(modality_by_notebook[notebook], "predictions")] = get_notebook_output(nb, "test-2_predictions")
    test_2["{}-{}".format(modality_by_notebook[notebook], "probabilities")] = get_notebook_output(nb, "test-2_probabilities")    

In [23]:
train.keys()

dict_keys(['features-labels', 'features-predictions', 'features-probabilities', 't2-labels', 't2-predictions', 't2-probabilities', 't1-labels', 't1-predictions', 't1-probabilities', 'ensemble-labels', 'ensemble-predictions', 'ensemble-probabilities'])

In [24]:
completed_notebooks

['evaluate-round-9-features-ebe1e4e9-fd1a-44c5-b06a-6c77579b5be5.ipynb',
 'evaluate-randomized-1-t2-59a36cd3-40b9-479f-9764-b7d8223264f7.ipynb',
 'evaluate-round-9-t1-acdd5165-089b-48fd-a936-af404577f7cf.ipynb',
 'evaluate-ensemble-ebe1e4e9-fd1a-44c5-b06a-6c77579b5be5-59a36cd3-40b9-479f-9764-b7d8223264f7-acdd5165-089b-48fd-a936-af404577f7cf.ipynb',
 'evaluate-ensemble-ebe1e4e9-fd1a-44c5-b06a-6c77579b5be5-59a36cd3-40b9-479f-9764-b7d8223264f7-acdd5165-089b-48fd-a936-af404577f7cf.ipynb']

In [25]:
import uuid
result = Result.query.filter(Result.uuid == UUIDS[0]).first()
train_set, validation_set, test_set = data(seed=uuid.UUID(result.split_seed), label_form=result.label_form, input_form=result.input_form, train_shuffle=False, test_shuffle=False, validation_shuffle=False, train_augment=False, validation_augment=False, test_augment=False)

  0%|          | 2/446 [00:00<03:04,  2.41it/s]


################################################################################
Exception occurred for: patient    Kidney-Penn-389
volume            0.658486
outcome             benign
age               0.689655
sex                      0
sort                 train
Name: Kidney-Penn-389, dtype: object
[Errno 2] No such file or directory: '/data/intrepidlemon/renal/20190617-preprocessed/Kidney-Penn-389-T1C-imagingVolume.nrrd'
Traceback (most recent call last):
  File "/home/user1/intrepidlemon/renal-mri/data.py", line 279, in generate_from_features
    t1_masked = load_image(t1_image_file, t1_seg_file, verbose=verbose)
  File "/home/user1/intrepidlemon/renal-mri/data.py", line 240, in load_image
    image, _ = nrrd.read(image_path)
  File "/home/user1/.local/share/virtualenvs/renal-mri-NanAjWL7/lib/python3.6/site-packages/nrrd/reader.py", line 427, in read
    with open(filename, 'rb') as fh:
FileNotFoundError: [Errno 2] No such file or directory: '/data/intrepidlemon/renal/20190617-p

  1%|▏         | 6/446 [00:01<01:41,  4.35it/s]


################################################################################
Exception occurred for: patient    Kidney-Penn-430
volume            0.658462
outcome             benign
age               0.735632
sex                      1
sort                 train
Name: Kidney-Penn-430, dtype: object
[Errno 2] No such file or directory: '/data/intrepidlemon/renal/20190617-preprocessed/Kidney-Penn-430-T1C-imagingVolume.nrrd'
Traceback (most recent call last):
  File "/home/user1/intrepidlemon/renal-mri/data.py", line 279, in generate_from_features
    t1_masked = load_image(t1_image_file, t1_seg_file, verbose=verbose)
  File "/home/user1/intrepidlemon/renal-mri/data.py", line 240, in load_image
    image, _ = nrrd.read(image_path)
  File "/home/user1/.local/share/virtualenvs/renal-mri-NanAjWL7/lib/python3.6/site-packages/nrrd/reader.py", line 427, in read
    with open(filename, 'rb') as fh:
FileNotFoundError: [Errno 2] No such file or directory: '/data/intrepidlemon/renal/20190617-p

 12%|█▏        | 55/446 [00:16<02:01,  3.21it/s]

...

 59%|█████▉    | 75/127 [00:30<00:13,  3.96it/s]


################################################################################
Exception occurred for: patient    Kidney-Penn-115
volume            0.658593
outcome          malignant
age               0.655172
sex                      0
sort            unassigned
Name: Kidney-Penn-115, dtype: object
[Errno 2] No such file or directory: '/data/intrepidlemon/renal/20190617-preprocessed/Kidney-Penn-115-T1C-imagingVolume.nrrd'
Traceback (most recent call last):
  File "/home/user1/intrepidlemon/renal-mri/data.py", line 279, in generate_from_features
    t1_masked = load_image(t1_image_file, t1_seg_file, verbose=verbose)
  File "/home/user1/intrepidlemon/renal-mri/data.py", line 240, in load_image
    image, _ = nrrd.read(image_path)
  File "/home/user1/.local/share/virtualenvs/renal-mri-NanAjWL7/lib/python3.6/site-packages/nrrd/reader.py", line 427, in read
    with open(filename, 'rb') as fh:
FileNotFoundError: [Errno 2] No such file or directory: '/data/intrepidlemon/renal/20190617-p

 66%|██████▌   | 84/127 [00:35<00:17,  2.42it/s]


################################################################################
Exception occurred for: patient    Kidney-Penn-266
volume            0.658453
outcome          malignant
age               0.632184
sex                      1
sort            unassigned
Name: Kidney-Penn-266, dtype: object
[Errno 2] No such file or directory: '/data/intrepidlemon/renal/20190617-preprocessed/Kidney-Penn-266-T1C-imagingVolume.nrrd'
Traceback (most recent call last):
  File "/home/user1/intrepidlemon/renal-mri/data.py", line 279, in generate_from_features
    t1_masked = load_image(t1_image_file, t1_seg_file, verbose=verbose)
  File "/home/user1/intrepidlemon/renal-mri/data.py", line 240, in load_image
    image, _ = nrrd.read(image_path)
  File "/home/user1/.local/share/virtualenvs/renal-mri-NanAjWL7/lib/python3.6/site-packages/nrrd/reader.py", line 427, in read
    with open(filename, 'rb') as fh:
FileNotFoundError: [Errno 2] No such file or directory: '/data/intrepidlemon/renal/20190617-p

100%|██████████| 63/63 [00:18<00:00,  4.74it/s]


In [26]:
MODALITIES = [
    "features",     
#    "t1-sensitive",
    "t2",
    "t1",
#    "t2-specific",
    "ensemble", 
]

MODALITY_KEY = {
    "features": "Clinical",     
    "t1": "T1C", 
    "t2": "T2WI", 
    "t1-sensitive": "T1C Sensitive", 
    "t2-sensitive": "T2WI Sensitive",     
    "t1-specific": "T1C Specific", 
    "t2-specific": "T2WI Specific",         
    "ensemble": "Ensemble", 
}

def get_pr_data_for_modality(dataset, comparison_models=[]): 
    results = list()
    points = list()
    for modality in MODALITIES: 
        labels = dataset["{}-labels".format(modality)]
        probabilities = dataset["{}-probabilities".format(modality)]
        predictions = dataset["{}-predictions".format(modality)]
        print(modality, len(labels), len(probabilities), len(predictions))
        acc = accuracy_score(labels, predictions)
        precision, recall, _ = precision_recall_curve(labels, probabilities)
        pr_auc = auc(recall, precision)
        stats = calculate_confusion_matrix_stats_predictions(labels, predictions)
        points.append({
            "modality": "{} (auc={:.2f}, acc={:.2f})".format(MODALITY_KEY[modality], pr_auc, acc),
            "precision": stats["PPV"][1],
            "recall": stats["TPR"][1],
        })
        for p, r in zip(precision, recall): 
            results.append({ "precision": p, "recall": r, "modality": "{} (auc={:.2f}, acc={:.2f})".format(MODALITY_KEY[modality], pr_auc, acc)})
    for probabilities in comparison_models: 
        modality = "Radiomics"
        labels = dataset["t1-labels"]
        predictions = [p > 0.5 for p in probabilities]
        print(modality, len(labels), len(probabilities), len(predictions))
        precision, recall, _ = precision_recall_curve(labels, probabilities)
        pr_auc = auc(recall, precision)
        stats = calculate_confusion_matrix_stats_predictions(labels, predictions)
        acc = accuracy_score(labels, predictions)
        points.append({
            "modality": "{} (auc={:.2f}, acc={:.2f})".format(MODALITY_KEY[modality], pr_auc, acc),
            "precision": stats["PPV"][1],
            "recall": stats["TPR"][1],
        })
        for p, r in zip(precision, recall): 
            results.append({ "precision": p, "recall": r, "modality": "{} (auc={:.2f}, acc={:.2f})".format(MODALITY_KEY[modality], pr_auc, acc)})           
    return results, pr_auc, []
        
def plot_multiple_precision_recall(dataset, experts=[], comparison_models=[]):
    results, auc, points = get_pr_data_for_modality(dataset, comparison_models)        
    if len(experts) > 0:
        for i, expert in enumerate(experts): 
            labels = dataset["t1-labels"]
            predictions = expert
            stats = calculate_confusion_matrix_stats_predictions(labels, predictions)
            acc = accuracy_score(labels, predictions)
            points.append({
                "precision": stats["PPV"][1],
                "recall": stats["TPR"][1],                
                "experts": "Expert {} (acc={:.2f})".format(i + 1, acc), 
            })
    fig, ax = plt.subplots()
    seaborn.lineplot(
        data=pandas.DataFrame(results),
        x="recall",
        y="precision",
        hue="modality",
        ax=ax, 
        err_style=None,
    )
    if points: 
        seaborn.scatterplot(
            data=pandas.DataFrame(points),
            x="recall",
            y="precision",
            hue="experts",
            style="experts",                        
            markers=["o", "v", "s", "P"],
            palette={ p["experts"]: "black" for p in points },            
            ax=ax,
        )
    ax.set_ylim(0.5, 1.04)
    ax.set_xlim(-0.04, 1.02)
    ax.legend(bbox_to_anchor=(1.05, 1), loc=2, borderaxespad=0.)
    return fig

def get_roc_data_for_modality(dataset, comparison_models=[]): 
    results = list()
    points = list()
    for modality in MODALITIES: 
        labels = dataset["{}-labels".format(modality)]
        probabilities = dataset["{}-probabilities".format(modality)]
        predictions = dataset["{}-predictions".format(modality)]
        fpr, tpr, _ = roc_curve(labels, probabilities, drop_intermediate=False)
        roc_auc = roc_auc_score(labels, probabilities)
        stats = calculate_confusion_matrix_stats_predictions(labels, predictions)
        acc = accuracy_score(labels, predictions)
        points.append({
            "modality": "{} (auc={:.2f}, acc={:.2f})".format(MODALITY_KEY[modality], roc_auc, acc),
            "fpr": stats["FPR"][1],
            "tpr": stats["TPR"][1],
        })
        for f, t in zip(fpr, tpr): 
            results.append({ "fpr": f, "tpr": t, "modality": "{} (auc={:.2f}, acc={:.2f})".format(MODALITY_KEY[modality], roc_auc, acc)})
    for probabilities in comparison_models: 
        modality = "Radiomics"
        labels = dataset["t1-labels"]
        predictions = [p > 0.5 for p in probabilities]
        fpr, tpr, _ = roc_curve(labels, probabilities, drop_intermediate=False)
        roc_auc = roc_auc_score(labels, probabilities)
        stats = calculate_confusion_matrix_stats_predictions(labels, predictions)
        acc = accuracy_score(labels, predictions)
        points.append({
            "modality": "{} (auc={:.2f}, acc={:.2f})".format(MODALITY_KEY[modality], roc_auc, acc),
            "fpr": stats["FPR"][1],
            "tpr": stats["TPR"][1],
        })
        for f, t in zip(fpr, tpr): 
            results.append({ "fpr": f, "tpr": t, "modality": "{} (auc={:.2f}, acc={:.2f})".format(MODALITY_KEY[modality], roc_auc, acc)})        
    return results, roc_auc, []
        
def plot_multiple_roc_curve(dataset, experts=[], comparison_models=[]):
    results, auc, points = get_roc_data_for_modality(dataset, comparison_models)
    if len(experts) > 0:
        for i, expert in enumerate(experts): 
            labels = dataset["t1-labels"]
            predictions = expert
            stats = calculate_confusion_matrix_stats_predictions(labels, predictions)
            acc = accuracy_score(labels, predictions)            
            points.append({
                "fpr": stats["FPR"][1],
                "tpr": stats["TPR"][1],
                "experts": "Expert {} (acc={:.2f})".format(i + 1, acc),                 
            })
    fig, ax = plt.subplots()
    seaborn.lineplot(
        data=pandas.DataFrame(results),
        x="fpr",
        y="tpr",
        hue="modality",
        ax=ax,
        err_style=None,
    )
    if points:     
        seaborn.scatterplot(
            data=pandas.DataFrame(points),
            x="fpr",
            y="tpr",
            hue="experts",
            style="experts",            
            ax=ax,
            markers=["o", "v", "s", "P"],
            palette={ p["experts"]: "black" for p in points },
        )
    ax.plot([0, 1], [0, 1], linestyle='--')
    ax.set_ylim(-0.04, 1.04)
    ax.set_xlim(-0.04, 1.02)
    ax.legend(bbox_to_anchor=(1.05, 1), loc=2, borderaxespad=0.)
    return fig

def get_statistics(dataset, experts=[], comparison_models=[]): 
    results = list()
    for modality in MODALITIES: 
        labels = dataset["{}-labels".format(modality)]
        probabilities = dataset["{}-probabilities".format(modality)]
        predictions = dataset["{}-predictions".format(modality)]
        roc_auc = roc_auc_score(labels, probabilities)
        precision, recall, _ = precision_recall_curve(labels, probabilities)
        pr_auc = auc(recall, precision)
        f1 = f1_score(labels, predictions)
        d = {
            "F1 Score": [f1, f1],            
            "ROC AUC": [roc_auc, roc_auc], 
            "PR AUC": [pr_auc, pr_auc],
            **calculate_confusion_matrix_stats_predictions(labels, predictions), 
            "Modality": [modality.capitalize(), modality.capitalize()], 
            "Total": [len(labels), len(labels)], 
            "Malignant": [len([l for l in labels if l]), len([l for l in labels if l])], 
            "Benign": [len([l for l in labels if not l]), len([l for l in labels if not l])], 
        } 
        # remove more that are not relevant to imbalanced datasets
        del d["TP"]
        del d["TN"]
        del d["FN"]
        del d["FP"]    
        del d["AM"]        
        del d["GM"]
        del d["FPR"]
        del d["FNR"]
        d["Acc (95% CI)"] = ["", "{:.2f} ({:.2f}-{:.2f})".format(d["Acc"][1], *adjusted_wald(d["Acc"][1], len(labels)))]
        d["TPR (95% CI)"] = ["", "{:.2f} ({:.2f}-{:.2f})".format(d["TPR"][1], *adjusted_wald(d["TPR"][1], len([l for l in labels if l])))]
        d["TNR (95% CI)"] = ["", "{:.2f} ({:.2f}-{:.2f})".format(d["TNR"][1], *adjusted_wald(d["TNR"][1], len([l for l in labels if not l])))]
        results.append(pandas.DataFrame(d).iloc[[1]])
    for i, expert in enumerate(experts): 
        labels = dataset["ensemble-labels"]
        predictions = expert
        f1 = f1_score(labels, predictions)        
        d = {
            "F1 Score": [f1, f1],            
            **calculate_confusion_matrix_stats_predictions(labels, predictions),             
            "Modality": ["Expert {}".format(i + 1), "Expert {}".format(i + 1)], 
            "Total": [len(labels), len(labels)], 
            "Malignant": [len([l for l in labels if l]), len([l for l in labels if l])], 
            "Benign": [len([l for l in labels if not l]), len([l for l in labels if not l])],             
        }
        # remove more that are not relevant to imbalanced datasets (remove from article too)
        del d["TP"]
        del d["TN"]
        del d["FN"]
        del d["FP"]               
        del d["AM"]        
        del d["GM"]
        del d["FPR"]
        del d["FNR"]        
        d["Acc (95% CI)"] = ["", "{:.2f} ({:.2f}-{:.2f})".format(d["Acc"][1], *adjusted_wald(d["Acc"][1], len(labels)))]
        d["TPR (95% CI)"] = ["", "{:.2f} ({:.2f}-{:.2f})".format(d["TPR"][1], *adjusted_wald(d["TPR"][1], len([l for l in labels if l])))]
        d["TNR (95% CI)"] = ["", "{:.2f} ({:.2f}-{:.2f})".format(d["TNR"][1], *adjusted_wald(d["TNR"][1], len([l for l in labels if not l])))]        
        results.append(pandas.DataFrame(d).iloc[[1]])
    for probabilities in comparison_models: 
        labels = dataset["ensemble-labels"]
        predictions = [p > 0.5 for p in probabilities]
        roc_auc = roc_auc_score(labels, probabilities)
        precision, recall, _ = precision_recall_curve(labels, probabilities)
        pr_auc = auc(recall, precision)
        f1 = f1_score(labels, predictions)
        d = {
            "F1 Score": [f1, f1],            
            "ROC AUC": [roc_auc, roc_auc], 
            "PR AUC": [pr_auc, pr_auc],
            **calculate_confusion_matrix_stats_predictions(labels, predictions), 
            "Modality": ["Radiomics", "Radiomics"], 
            "Total": [len(labels), len(labels)], 
            "Malignant": [len([l for l in labels if l]), len([l for l in labels if l])], 
            "Benign": [len([l for l in labels if not l]), len([l for l in labels if not l])],             
        } 
        # remove more that are not relevant to imbalanced datasets (remove from article too)
        del d["TP"]
        del d["TN"]
        del d["FN"]
        del d["FP"]         
        del d["AM"]        
        del d["GM"]
        del d["FPR"]
        del d["FNR"]
        d["Acc (95% CI)"] = ["", "{:.2f} ({:.2f}-{:.2f})".format(d["Acc"][1], *adjusted_wald(d["Acc"][1], len(labels)))]
        d["TPR (95% CI)"] = ["", "{:.2f} ({:.2f}-{:.2f})".format(d["TPR"][1], *adjusted_wald(d["TPR"][1], len([l for l in labels if l])))]
        d["TNR (95% CI)"] = ["", "{:.2f} ({:.2f}-{:.2f})".format(d["TNR"][1], *adjusted_wald(d["TNR"][1], len([l for l in labels if not l])))]        
        results.append(pandas.DataFrame(d).iloc[[1]])

    return pandas.concat(results, axis=0, sort=False).set_index("Modality")

def get_experts_for_names(features, names, experts=["expert1", "expert2"], transform=int, default=0): 
    result = list()
    for e in experts: 
        expert_results = list()
        for n in names: 
            f = features.get(n, None)
            if f is None: 
                print("error, cannot find {}".format(n))
                expert_results.append(default)
                continue
            r = f.get(e, default)
            if r == "": 
                r = 0
            r = transform(r)
            expert_results.append(r)
        result.append(expert_results)
    return result

In [27]:
expert_features = all_features(files = [config.EXPERTS])
comparison_model_features = all_features(files=["features/comparison-models.csv"])
df = get_statistics(train, get_experts_for_names(expert_features, train_set.names), get_experts_for_names(comparison_model_features, train_set.names, experts=["radiomics"], transform=float))
df = df.drop(['Acc', "TPR", "TNR", "Total", "Malignant", "Benign"], axis=1)
print()
print("##### Train")
print()
print(tabulate(df, tablefmt="pipe", headers="keys", floatfmt="#.3g").replace("nan", "N/A").replace("Features", "Clinical"))
df = get_statistics(validation, get_experts_for_names(expert_features, validation_set.names), get_experts_for_names(comparison_model_features, validation_set.names, experts=["radiomics"], transform=float))
df = df.drop(['Acc', "TPR", "TNR", "Total", "Malignant", "Benign"], axis=1)
print()
print("##### Validation")
print()
print(tabulate(df, tablefmt="pipe", headers="keys", floatfmt="#.3g").replace("nan", "N/A").replace("Features", "Clinical"))
df = get_statistics(test, get_experts_for_names(expert_features, test_set.names, experts=["expert1", "expert2", "expert3", "expert4"]), get_experts_for_names(comparison_model_features, test_set.names, experts=["radiomics"], transform=float))
test_statistics = df
df = df.drop(['Acc', "TPR", "TNR", "Total", "Malignant", "Benign"], axis=1)
print()
print("##### Test")
print()
print(tabulate(df, tablefmt="pipe", headers="keys", floatfmt="#.2g").replace("nan", "N/A").replace("Features", "Clinical"))
#df = get_statistics(test_1, get_experts_for_names(expert_features, test_set.names, experts=["expert1", "expert2", "expert3", "expert4"]), get_experts_for_names(comparison_model_features, test_set.names, experts=["radiomics"], transform=float))
#test_1_statistics = df
#df = df.drop(['Acc', "TPR", "TNR", "Total", "Malignant", "Benign"], axis=1)
#print()
#print("##### Test 1")
#print()
#print(tabulate(df, tablefmt="pipe", headers="keys", floatfmt="#.2g").replace("nan", "N/A").replace("Features", "Clinical"))
#df = get_statistics(test_2, get_experts_for_names(expert_features, test_set.names, experts=["expert1", "expert2", "expert3", "expert4"]), get_experts_for_names(comparison_model_features, test_set.names, experts=["radiomics"], transform=float))
#test_2_statistics = df
#df = df.drop(['Acc', "TPR", "TNR", "Total", "Malignant", "Benign"], axis=1)
#print()
#print("##### Test 2")
#print()
#print(tabulate(df, tablefmt="pipe", headers="keys", floatfmt="#.2g").replace("nan", "N/A").replace("Features", "Clinical"))


error, cannot find Kidney-CHOP-061
error, cannot find Kidney-CHOP-062
error, cannot find Kidney-CHOP-092
error, cannot find Kidney-CHOP-074
error, cannot find Kidney-Penn-380
error, cannot find Kidney-Penn-411
error, cannot find Kidney-Penn-413
error, cannot find Kidney-CHOP-045
error, cannot find Kidney-CHOP-010
error, cannot find Kidney-CHOP-065
error, cannot find Kidney-Penn-402
error, cannot find Kidney-CHOP-060
error, cannot find Kidney-CHOP-042
error, cannot find Kidney-CHOP-026
error, cannot find Kidney-CHOP-067
error, cannot find Kidney-CHOP-014
error, cannot find Kidney-CHOP-057
error, cannot find Kidney-CHOP-019
error, cannot find Kidney-CHOP-063
error, cannot find Kidney-Penn-420
error, cannot find Kidney-CHOP-021
error, cannot find Kidney-CHOP-031
error, cannot find Kidney-CHOP-106
error, cannot find Kidney-CHOP-020
error, cannot find Kidney-CHOP-072
error, cannot find Kidney-CHOP-064
error, cannot find Kidney-Penn-388
error, cannot find Kidney-CHOP-091
error, cannot find K

In [28]:
cross_validation_statistics = {
 'F1 Score': 0.88,
 'ROC AUC': 0.6243243243243244,
 'PR AUC': 0.90,
 'Acc': 0.80,
 'TPR': 0.96,
 'TNR': 0.22,
 'PPV': 0.896551724137931,
 'NPV': 0.3888888888888889,
 'FDR': 0.10344827586206896,
 'Total': 47,
 'Malignant': 37,
 'Benign': 10,
 'Acc (95% CI)': '0.70 (0.56-0.81)',
 'TPR (95% CI)': '0.70 (0.54-0.83)',
 'TNR (95% CI)': '0.70 (0.39-0.90)'
}

In [29]:
CI_text = """{value:.2g} (95% CI: {low:.2g}-{high:.2g})"""

statistics_text = """test accuracy of {accuracy}, F1 score of {f1:.2g}, and sensitivity of {sensitivity}, and specificity of {specificity}"""
def format_statistics(data):
    acc_low, acc_high = adjusted_wald(data["Acc"], data["Total"])
    tpr_low, tpr_high = adjusted_wald(data["TPR"], data["Malignant"])
    tnr_low, tnr_high = adjusted_wald(data["TNR"], data["Benign"])
    data_dict = {
        "accuracy": CI_text.format(value=data["Acc"], low=acc_low, high=acc_high),
        "sensitivity": CI_text.format(value=data["TPR"], low=tpr_low, high=tpr_high),
        "specificity": CI_text.format(value=data["TNR"], low=tnr_low, high=tnr_high),
        "f1": data["F1 Score"],
    }
    return statistics_text.format(**data_dict)

statistics_AUC_text = """test accuracy of {accuracy}, F1 score of {f1:.2g}, precision recall AUC of {PR_AUC:.2g}, sensitivity of {sensitivity}, and specificity of {specificity}"""
def format_AUC_statistics(data):
    acc_low, acc_high = adjusted_wald(data["Acc"], data["Total"])
    tpr_low, tpr_high = adjusted_wald(data["TPR"], data["Malignant"])
    tnr_low, tnr_high = adjusted_wald(data["TNR"], data["Benign"])
    data_dict = {
        "accuracy": CI_text.format(value=data["Acc"], low=acc_low, high=acc_high),
        "sensitivity": CI_text.format(value=data["TPR"], low=tpr_low, high=tpr_high),
        "specificity": CI_text.format(value=data["TNR"], low=tnr_low, high=tnr_high),
        "f1": data["F1 Score"],
        "PR_AUC": data["PR AUC"],
    }
    return statistics_AUC_text.format(**data_dict)

comparison_text ="""{accuracy_judge} test accuracy ({accuracy_1:.2g} vs. {accuracy_2:.2g}, p={accuracy_p:.2g}), {sensitivity_judge} test sensitivity ({sensitivity_1:.2g} vs. {sensitivity_2:.2g}, p={sensitivity_p:.2g}) and {specificity_judge} test specificity ({specificity_1:.2g} vs. {specificity_2:.2g}, p={specificity_p:.2g})"""
def format_comparison_text(data_1, data_2):
    data_dict = {
        "accuracy_judge": "higher" if data_1["Acc"] > data_2["Acc"] else "lower",
        "sensitivity_judge": "higher" if data_1["TPR"] > data_2["TPR"] else "lower",
        "specificity_judge": "higher" if data_1["TNR"] > data_2["TNR"] else "lower",
        "accuracy_1": data_1["Acc"],
        "specificity_1": data_1["TNR"],
        "sensitivity_1": data_1["TPR"],
        "accuracy_2": data_2["Acc"],
        "specificity_2": data_2["TNR"],
        "sensitivity_2": data_2["TPR"],
        "accuracy_p": binom_test(int(data_1["Acc"] * data_1["Total"]), data_1["Total"], data_2["Acc"]),
        "specificity_p": binom_test(int(data_1["TNR"] * data_1["Benign"]), data_1["Benign"], data_2["TNR"]),
        "sensitivity_p": binom_test(int(data_1["TPR"] * data_1["Malignant"]), data_1["Malignant"], data_2["TPR"]),
    }
    return comparison_text.format(**data_dict)

results_text = """
## Model performance
The train, validation and test sets were balanced in terms of age, gender, tumor size, tumor laterality, tumor location, histologic diagnosis, and institution (Supplemental table S1).

Performance characteristics of the clinical variable logistic regression, models trained on T1C and T2WI images, and the final ensemble model in test set are summarized in Table 2. Performance characteristics in training and validation sets are summarized in Supplemental table S2. The clinical variable logistic regression achieved a {logistic_regression_statistics}.

The T1C trained model achieved a {t1_statistics}. The T2WI trained model achieved a {t2_statistics}. The ensemble model achieved a {ensemble_statistics}. The ensemble model achieved comparative performance on the second set of test set segmentations with a {ensemble_1_statistics} and the third set of test set segmentation with a {ensemble_2_statistics} (Supplemental table S3). On average, cross validation analysis of the ensemble model demonstrated a {cross_validation_statistics}. Supplemental table S4 summarizes the cross-validation performance of the ensemble model.

In comparison, expert 1 achieved a {expert_1_statistics}; expert 2 had a {expert_2_statistics}; expert 3 had a {expert_3_statistics}; expert 4 had a {expert_4_statistics}. Radiomics model achieved a {radiomics_statistics} (Supplemental table S5).

CONFIRM THE BELOW PARAGRAPH IS STILL TRUE

Compared to all experts averaged, the ensemble deep learning model had {ensemble_vs_experts}, COMMENT although none of these statistics was significantly different. Compared to the radiomics model, the ensemble deep learning model had {ensemble_vs_radiomics}; COMMENT difference in accuracy was not significant but differences in sensitivity and specificity between radiomics and ensemble deep learning models were significant. Compared to all experts averaged, the radiomics model had {radiomics_vs_experts}; COMMENT difference in accuracy was not significant, but differences in sensitivity and specificity between radiomics and averaged expert performance metrics were significant. Figure 2 shows the precision recall curves of all models overlaid with expert performance.

Grad-Cam focus maps demonstrate that for the correctly classified images by our model, the algorithm was more likely to focus on the lesion or part of the lesion deemed important by the radiologists, while focus was more likely to be on the surrounding tissue for the incorrectly classified lesions (Figure 3). t-SNE representation of the final dense layer of ResNet demonstrates good separation of malignant and benign lesions by the model when compared to histopathological diagnosis (Figure 4).

"""

print(results_text.format(
    logistic_regression_statistics=format_AUC_statistics(dict(test_statistics.loc["Features"])),
    t1_statistics=format_AUC_statistics(dict(test_statistics.loc["T1"])),
    t2_statistics=format_AUC_statistics(dict(test_statistics.loc["T2"])),
    ensemble_statistics=format_AUC_statistics(dict(test_statistics.loc["Ensemble"])),
    ensemble_1_statistics=format_AUC_statistics(dict(test_1_statistics.loc["Ensemble"])),
    ensemble_2_statistics=format_AUC_statistics(dict(test_2_statistics.loc["Ensemble"])),
    cross_validation_statistics=format_AUC_statistics(cross_validation_statistics),
    expert_1_statistics=format_statistics(dict(test_statistics.loc["Expert 1"])),
    expert_2_statistics=format_statistics(dict(test_statistics.loc["Expert 2"])),
    expert_3_statistics=format_statistics(dict(test_statistics.loc["Expert 3"])),
    expert_4_statistics=format_statistics(dict(test_statistics.loc["Expert 4"])),
    radiomics_statistics=format_statistics(dict(test_statistics.loc["Radiomics"])),
    ensemble_vs_experts=format_comparison_text(dict(test_statistics.loc["Ensemble"]), dict(test_statistics.loc[["Expert 1", "Expert 2", "Expert 3", "Expert 4"]].mean())),
    ensemble_vs_radiomics=format_comparison_text(dict(test_statistics.loc["Ensemble"]), dict(test_statistics.loc["Radiomics"])),
    radiomics_vs_experts=format_comparison_text(dict(test_statistics.loc["Radiomics"]), dict(test_statistics.loc[["Expert 1", "Expert 2", "Expert 3", "Expert 4"]].mean())),
))

NameError: name 'test_1_statistics' is not defined

In [ ]:
results_text = """
## Inter-segmenter agreement

The average DSC among our three segmenters was {dice_average}. Segmenter 1 and 2 had an average DSC of {dice_1_2}. Segmenter 2 and 3 had an average DSC of {dice_2_3}. Segmenter 1 and 3 had an average DSC of {dice_1_3}. Benign lesions had an average DSC of {dice_benign} across all reviewers, while malignant lesions had an average DSC of {dice_malignant} across all reviewers. 

## Model performance
The train, validation and test sets were balanced in terms of age, gender, tumor size, tumor laterality, tumor location, histologic diagnosis, and institution (Supplemental table S1). 

Performance characteristics of the clinical variable logistic regression, models trained on T1C and T2WI images, and the final ensemble model in test set are summarized in Table 2. Performance characteristics in training and validation sets are summarized in Supplemental table S2. The clinical variable logistic regression achieved a test accuracy of {regression_accuracy}, F1 score of {regression_F1}, precision recall AUC of {regression_PR_AUC}, sensitivity of {regression_sensitivity}, and specificity of {regression_specificity}. 

The T1C trained model achieved a test accuracy of {t1c_accuracy}, F1 score of {t1c_f1}, precision recall AUC of {t1c_PR_AUC}, sensitivity of 0.92 (95% CI: 0.78-0.98), and specificity of 0.50 (95% CI: 0.24-0.76). The T2WI trained model achieved a test accuracy of 0.83 (95% CI: 0.70-0.91), F1 score of 0.90, precision recall AUC of 0.92, sensitivity of 0.97 (95% CI: 0.84-1.00), and specificity of 0.30 (95% CI: 0.10-0.61). The ensemble model achieved a test accuracy of 0.89 (95% CI: 0.77-0.96), F1 score of 0.94, precision recall AUC of 0.90, sensitivity of 1.0 (95% CI: 0.88-1.00), and specificity of 0.50 (95% CI: 0.24-0.76). The ensemble model achieved comparative performance on the second set of test set segmentations with an accuracy of 0.81 (95% CI: 0.67-0.90), F1 score of 0.89, precision recall AUC of 0.89, sensitivity of 0.97 (95% CI: 0.84-1.00), and specificity of 0.20 (95% CI: 0.05-0.52) and the third set of test set segmentation with an accuracy of 0.81 (95% CI: 0.67-0.90), F1 score of 0.89, precision recall AUC of 0.83, sensitivity of 1.0 (95% CI: 0.88-1.00), and specificity of 0.10 (95% CI: 0.00-0.42) (Supplemental table S3). The average cross validation test accuracy was 0.80 (95% CI: 0.66-0.89) with average F1 score of 0.88, precision recall AUC of 0.90, sensitivity of 0.96 (95% CI: 0.83-1.00), and specificity of 0.22 (95% CI: 0.06-0.54). Supplemental table S4 summarizes the cross-validation performance of the ensemble model.

In comparison, expert 1 achieved a test accuracy of 0.85 (95% CI: 0.72-0.92), F1 score of 0.91, and sensitivity of 1.0 (95% CI: 0.88-1.00), and specificity of 0.30 (95% CI: 0.10-0.61); expert 2 had a test accuracy of 0.79 (95% CI: 0.65-0.88), F1 score of 0.88, and sensitivity of 0.95 (95% CI: 0.82-1.00), and specificity of 0.20 (95% CI: 0.05-0.52); expert 3 had a test accuracy of 0.74 (95% CI: 0.60-0.84), F1 score of 0.73, sensitivity of 0.81 (95% CI: 0.65-0.90), and specificity of 0.50 (95% CI: 0.24-0.76); expert 4 had a test accuracy of 0.85 (95% CI: 0.72-0.93), F1 score of 0.91, and sensitivity of 0.92 (95% CI: 0.78 -0.98), and specificity of 0.60 (95% CI: 0.31-0.83). Radiomics model achieved a test accuracy of 0.78 (95% CI: 0.64-0.88), F1 score of 0.84, sensitivity of 0.76 (95% CI: 0.60-0.87), and specificity of 0.80 (95% CI: 0.47-0.95) (Supplemental table S5).

Compared to all experts averaged, the ensemble deep learning model had higher test accuracy (0.89 vs. 0.81, p=0.32), higher test sensitivity (1.00 vs. 0.92, p=0.07) and higher test specificity (0.50 vs. 0.40, p=0.53), although none of these statistics was significantly different. Compared to the radiomics model, the ensemble deep learning model had higher test accuracy (0.89 vs. 0.77, p=0.12), higher test sensitivity (1.00 vs. 0.76, p<0.01) and lower test specificity (0.50 vs. 0.80, p=0.03); difference in accuracy was not significant but differences in sensitivity and specificity between radiomics and ensemble deep learning models were significant. Compared to all experts averaged, the radiomics model had lower test accuracy (0.77 vs. 0.81, p=0.61), lower test sensitivity (0.76 vs. 0.92, p=0.0021) and higher test specificity (0.80 vs. 0.40, p<0.01); difference in accuracy was not significant, but differences in sensitivity and specificity between radiomics and averaged expert performance metrics were significant. Figure 2 shows the precision recall curves of all models overlaid with expert performance. 

Grad-Cam focus maps demonstrate that for the correctly classified images by our model, the algorithm was more likely to focus on the lesion or part of the lesion deemed important by the radiologists, while focus was more likely to be on the surrounding tissue for the incorrectly classified lesions (Figure 3). t-SNE representation of the final dense layer of ResNet demonstrates good separation of malignant and benign lesions by the model when compared to histopathological diagnosis (Figure 4).

"""

In [ ]:
train

In [ ]:
fig = plot_multiple_precision_recall(train, experts=get_experts_for_names(expert_features, train_set.names))#, comparison_models=get_experts_for_names(comparison_model_features, train_set.names, experts=['radiomics'], transform=float))
fig.savefig("figures/combined-train-precision-recall.svg", bbox_inches = "tight")
fig = plot_multiple_precision_recall(validation, experts=get_experts_for_names(expert_features, validation_set.names))#,  comparison_models=get_experts_for_names(comparison_model_features, validation_set.names, experts=['radiomics'], transform=float))
fig.savefig("figures/combined-validation-precision-recall.svg", bbox_inches = "tight")
fig = plot_multiple_precision_recall(test, experts=get_experts_for_names(expert_features, test_set.names, experts=["expert1", "expert2", "expert3", "expert4"]))#, comparison_models=get_experts_for_names(comparison_model_features, test_set.names, experts=['radiomics'], transform=float))
fig.savefig("figures/combined-test-precision-recall.svg", bbox_inches = "tight")
fig = plot_multiple_precision_recall(test_1, experts=get_experts_for_names(expert_features, test_set.names, experts=["expert1", "expert2", "expert3", "expert4"]))#, comparison_models=get_experts_for_names(comparison_model_features, test_set.names, experts=['radiomics'], transform=float))
fig.savefig("figures/combined-test-1-precision-recall.svg", bbox_inches = "tight")
fig = plot_multiple_precision_recall(test_2, experts=get_experts_for_names(expert_features, test_set.names, experts=["expert1", "expert2", "expert3", "expert4"]))#, comparison_models=get_experts_for_names(comparison_model_features, test_set.names, experts=['radiomics'], transform=float))
fig.savefig("figures/combined-test-2-precision-recall.svg", bbox_inches = "tight")
fig = plot_multiple_roc_curve(train, experts=get_experts_for_names(expert_features, train_set.names))#, comparison_models=get_experts_for_names(comparison_model_features, train_set.names, experts=['radiomics'], transform=float))
fig.savefig("figures/combined-train-roc.svg", bbox_inches = "tight")
fig = plot_multiple_roc_curve(validation, experts=get_experts_for_names(expert_features, validation_set.names))#, comparison_models=get_experts_for_names(comparison_model_features, validation_set.names, experts=['radiomics'], transform=float))
fig.savefig("figures/combined-validation-roc.svg", bbox_inches = "tight")
fig = plot_multiple_roc_curve(test, experts=get_experts_for_names(expert_features, test_set.names, experts=["expert1", "expert2", "expert3", "expert4"]))#, comparison_models=get_experts_for_names(comparison_model_features, test_set.names, experts=['radiomics'], transform=float))
fig.savefig("figures/combined-test-roc.svg", bbox_inches = "tight")
fig = plot_multiple_roc_curve(test_1, experts=get_experts_for_names(expert_features, test_set.names, experts=["expert1", "expert2", "expert3", "expert4"]))#, comparison_models=get_experts_for_names(comparison_model_features, test_set.names, experts=['radiomics'], transform=float))
fig.savefig("figures/combined-test-1-roc.svg", bbox_inches = "tight")
fig = plot_multiple_roc_curve(test_2, experts=get_experts_for_names(expert_features, test_set.names, experts=["expert1", "expert2", "expert3", "expert4"]))#, comparison_models=get_experts_for_names(comparison_model_features, test_set.names, experts=['radiomics'], transform=float))
fig.savefig("figures/combined-test-2-roc.svg", bbox_inches = "tight")

In [ ]:
def correct_num_for_modality(dataset, experts=[], comparison_models=[]): 
    results = list()
    if len(experts) > 0:
        for i, expert in enumerate(experts): 
            labels = dataset["t1-labels"]
            predictions = expert
            correct = sum([ labels[i] == p for i, p in enumerate(predictions) ])
            total = len(labels)
            results.append({ 
                "correct": correct, 
                "total": total, 
                "incorrect": total-correct, 
                "modality": "Expert {}".format(i),
                "acc": correct/total,                
                "wald": adjusted_wald(correct/total, total),
                **calculate_confusion_matrix_stats_predictions(labels, predictions),
            })
    for modality in MODALITIES: 
        labels = dataset["{}-labels".format(modality)]
        probabilities = dataset["{}-probabilities".format(modality)]
        predictions = dataset["{}-predictions".format(modality)]
        total = len(labels)
        correct = sum([ labels[i] == p for i, p in enumerate(predictions) ])
        results.append({ 
            "correct": correct, 
            "total": total, 
            "incorrect": total-correct, 
            "modality": MODALITY_KEY[modality],
            "acc": correct/total,
            "wald": adjusted_wald(correct/total, total),
            **calculate_confusion_matrix_stats_predictions(labels, predictions),            
        })        
    for probabilities in comparison_models: 
        modality = "Radiomics"
        labels = dataset["t1-labels"]
        predictions = [p > 0.5 for p in probabilities]
        predictions = dataset["{}-predictions".format(modality)]
        total = len(labels)
        correct = sum([ labels[i] == p for i, p in enumerate(predictions) ])
        results.append({ 
            "correct": correct, 
            "total": total, 
            "incorrect": total-correct, 
            "modality": modality,
            "acc": correct/total,
            "wald": adjusted_wald(correct/total, total),
            **calculate_confusion_matrix_stats_predictions(labels, predictions),            
        })                
    return results

In [ ]:
results = correct_num_for_modality(test, experts=get_experts_for_names(expert_features, test_set.names, experts=["expert1", "expert2", "expert3", "expert4"]))#, comparison_models=get_experts_for_names(comparison_model_features, test_set.names, experts=['radiomics'], transform=float))

In [ ]:
experts = get_experts_for_names(expert_features, test_set.names, experts=["expert1", "expert2", "expert3", "expert4"])

In [ ]:
kappa_scores = list()
for e in experts: 
    current = list()
    for x in experts: 
        current.append(cohen_kappa_score(e, x))
    kappa_scores.append(current)
kappa_scores

In [ ]:
seaborn.heatmap(kappa_scores, annot=True, square=True, fmt=".2f", xticklabels=["1", "2", "3", "4"], yticklabels=["1", "2", "3", "4"])

In [ ]:
from itertools import combinations
import numpy as np

In [ ]:
unique_kappa = list()
for x, y in combinations(experts, 2): 
    unique_kappa.append(cohen_kappa_score(x, y))
np.average(unique_kappa)

In [ ]:
def fleiss_kappa(ratings, n):
    '''
    Computes the Fleiss' kappa measure for assessing the reliability of 
    agreement between a fixed number n of raters when assigning categorical
    ratings to a number of items.
    
    Args:
        ratings: a list of (item, category)-ratings
        n: number of raters
        k: number of categories
    Returns:
        the Fleiss' kappa score
    
    See also:
        http://en.wikipedia.org/wiki/Fleiss'_kappa
    '''
    items = set()
    categories = set()
    n_ij = {}
    
    for i, c in ratings:
        items.add(i)
        categories.add(c)
        n_ij[(i,c)] = n_ij.get((i,c), 0) + 1
    
    N = len(items)
    
    p_j = dict(((c, sum(n_ij.get((i, c), 0) for i in items) / (1.0 * n * N)) for c in categories))
    P_i = dict(((i, (sum(n_ij.get((i, c), 0) ** 2 for c in categories) - n) / (n * (n - 1.0))) for i in items))

    P_bar = sum(P_i.values()) / (1.0 * N)
    P_e_bar = sum(value ** 2 for value in p_j.values())
    
    kappa = (P_bar - P_e_bar) / (1 - P_e_bar)
    
    return kappa

In [ ]:
fleiss_kappa_array = list()
for e in list(zip(*experts)): 
    for i in e: 
        fleiss_kappa_array.append((1, i))

In [ ]:
fleiss_kappa(fleiss_kappa_array, 4)

In [ ]:
ratings = [(1, 'yes')] * 10 + [(1, 'no')] * 0  + \
[(2, 'yes')] * 8  + [(2, 'no')] * 2  + \
[(3, 'yes')] * 9  + [(3, 'no')] * 1  + \
[(4, 'yes')] * 0  + [(4, 'no')] * 10 + \
[(5, 'yes')] * 7  + [(5, 'no')] * 3
ratings

In [ ]:
x = np.array(experts).T
fleiss_kappa(x)

In [ ]:
ensemble = results.pop()
for r in results: 
    print(r["modality"], fisher_exact([[ensemble["correct"], r["correct"]], [ensemble["incorrect"], r["incorrect"]]]))

In [ ]:
results

In [ ]:
from IPython.display import HTML
HTML('''<script>
code_show=true; 
function code_toggle() {
 if (code_show){
 $('div.input').hide();
 } else {
 $('div.input').show();
 }
 code_show = !code_show
} 
$( document ).ready(code_toggle);
</script>
The raw code for this IPython notebook is by default hidden for easier reading.
To toggle on/off the raw code, click <a href="javascript:code_toggle()">here</a>.''')